In [133]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
from sklearn.cluster import OPTICS, cluster_optics_dbscan

In [134]:
model = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L3-v2')


In [135]:
data = pd.read_csv("../data/location_data.csv",index_col=None)

In [60]:
data.describe()

,city,road,house_number,postcode,state
count,63695,63695,63695,63695,63695
unique,5534,17184,6353,11015,121
top,houston,main st,101,46227,ca
freq,640,599,357,33,3251


In [61]:
data.head()

,city,road,house_number,postcode,state
0,fort worth,sycamore school rd,5421,76123,texas
1,gautier,highway 90,2701,39553,mississippi
2,farmington,e main,4750,87401,nm
3,richland,queensgate drive,2751,99352,washington
4,davie,s state hwy seven,1600,33317,fl


In [62]:
data.shape

(63695, 5)

In [63]:
data_no_dup = data.drop_duplicates(ignore_index=True)
data_no_dup.shape # we have many duplicate rows

(41423, 5)

In [64]:
data_no_dup # we're keeping the old indexes

,city,road,house_number,postcode,state
0,fort worth,sycamore school rd,5421,76123,texas
1,gautier,highway 90,2701,39553,mississippi
2,farmington,e main,4750,87401,nm
3,richland,queensgate drive,2751,99352,washington
4,davie,s state hwy seven,1600,33317,fl
...,...,...,...,...,...
41418,highland,baseline st,27774,92346,california
41419,hilliard,main st,3685,43026,ohio
41420,window rock,window rock loop road,264,86515,az
41421,denver,s santa fe dr,1395,80223,co


In [136]:
# sentences = data_no_dup["city"]+" "+ data_no_dup["road"] +" "+ data_no_dup["house_number"]\
#             +" "+ data_no_dup["postcode"]+" "+ data_no_dup["state"]
sentences = data_no_dup["city"]+" "+ data_no_dup["road"] +" "+ data_no_dup["postcode"]\
                +" "+ data_no_dup["state"]

In [175]:
sentences[34357] # this doesn't seem to be an address in the USA...
# this will create extra difficulties, and we may need to use a multi-lingual model

'kota jkt utara jl. boulevard artha gading jalan kav no.01 rt.18/rw.8 klp. gading bar. kec. klp. gading 14240 daerah khusus ibukota jakarta'

In [157]:
sentence_embeddings = model.encode(sentences)
sentence_embeddings = pd.DataFrame(sentence_embeddings)
# this will not scale well...

In [21]:
# sentence_embeddings.to_csv("../data/embeddings.csv")
from operator import index


sentence_embeddings = pd.read_csv("../data/embeddings.csv", index_col = 0)

In [170]:
clustering = OPTICS(min_samples=2,
                    n_jobs = -1,xi = 0.00001,p=30).fit(sentence_embeddings[:100])

In [173]:
clustering.labels_

array([ 0,  7, -1, -1,  3,  8, -1,  0,  4,  5,  6, -1,  4, -1,  1,  9, 11,
        8, -1,  3,  5, -1, 10, -1, -1,  3, -1, -1, -1, -1, -1,  3,  9, -1,
       -1,  1,  7, 11,  7, -1, -1, -1, 13, -1, -1, -1,  0,  7, -1,  9, -1,
       -1, -1,  6, -1,  8, 10, -1,  1, 12,  1, -1, -1, -1, -1,  2,  8, -1,
       12, -1, -1, -1, -1, -1, 11, -1,  3, -1,  3,  7, -1, -1, -1, -1, -1,
        9, 13, -1,  0, -1,  0, -1, -1, -1, -1, -1, -1,  2, -1,  4])

In [176]:
index = sentence_embeddings[:100][clustering.labels_==7].index

for id in index:
    print(sentences[:100][id])


gautier highway 90 39553 mississippi
phenix city us highway 80 36867 alabama
urbandale 86th st 50322 iowa
oakland alameda ave 94601 ca
visalia s mooney blvd 93277 ca


In [177]:
for id in index:
    print(id)

1
36
38
47
79
